In [1]:
import sys
print(sys.executable)

/home/kwj/team_proj/faiss_2/bin/python3


In [2]:
import sys
print(sys.version)

3.8.20 (default, Sep  7 2024, 18:35:08) 
[GCC 11.4.0]


In [ ]:
!pip install fitz frontend pymupdf pypdf dotenv langchain torch langchain_community transformers datasets
!pip install "numpy<2"
!pip install --upgrade pybind11
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 11.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [3]:
import numpy as np
print(np.__version__)

1.24.4


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os

# path = "C:\\KIMUJUNG\\team_project\\team_project3\\pdf_folder"
path = "pdf_folder"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

file_list: ['일반_let safe 팬덤안심 상해보험.pdf', '일반_let safe 골프보험.pdf', '일반_let safe 주택화재보험Ⅱ.pdf', '공동인수_영업용 자동차보험(공동).pdf', '상해,질병_(무) let care 실손의료보험Ⅳ(개인 단체연계용)(2501).pdf', '상해,질병_(무) 암보험(2502).pdf', '이륜차_let way 이륜자동차보험.pdf', '일반_let safe 레저상해보험.pdf', '상해,질병_(무) let smile 종합건강보험(더끌림 포우먼)(2501) 1~2종.pdf', '일반_let safe 시민안전보험.pdf', '개인용_let click 개인용자동차보험(인터넷).pdf', '일반_let safe 원게임골프보험.pdf', '상해,질병_(무) 미니암보험(2404).pdf', '상해,질병_(무) 입원비보험(2409).pdf', '일반_let discover 국내여행보험Ⅱ.pdf', '업무용_let way 업무용법인소유자동차보험.pdf', '일반_let safe 실손의료보험(해외여행).pdf', '상해,질병_(무) let simple 간편311 건강보험(2501).pdf', '연금보험_let care 연금저축손해보험Ⅰ(2401).pdf', '상해,질병_(무) let play 종합건강보험(토닥토닥)(2501).pdf', '일반_let safe 실손의료보험(국내여행).pdf', '기타_let way 자동차취급업자종합보험.pdf', '일반_주택 임차보증금반환 민사소송 및 강제집행 변호사선임비용 보장보험.pdf', '상해,질병_(무) 직장인보험(2501).pdf', '제도성특약_4세대 실손의료비보장 계약전환제도(2021) 특별약관.pdf', '운전자_(무) 운전자보험(2501).pdf', '일반_let safe 주택화재보험.pdf', '상해,질병_(무) let smile 종합건강보험(더채움 베이직)(2501).pdf', '일반_let safe 상해보험.pdf', '일반_let safe 제품보증연장보험(간단

In [1]:
import os
import faiss
import pickle
import numpy as np
import torch
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer
import fitz  # PyMuPDF

# 1️⃣ GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/home/kwj/team_proj/faiss_2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
# path = "C:\\KIMUJUNG\\team_project\\team_project3\\pdf_folder"
path = "pdf_folder"
file_list = os.listdir(path)

In [3]:

# 2️⃣ 모델 및 토크나이저 로드 (GPU로 이동)
model_name = "kakaocorp/kanana-nano-2.1b-embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).to(device)

# 3️⃣ 임베딩 생성 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # 모든 입력 텐서를 GPU로 이동
    inputs = {key: val.to(device) for key, val in inputs.items()}
    pool_mask = torch.ones(inputs["input_ids"].shape, dtype=torch.long).to(device)

    with torch.no_grad():
        outputs = model(**inputs, pool_mask=pool_mask)

    return outputs.embedding.cpu().numpy()  # GPU에서 CPU로 변환 후 반환

# 4️⃣ 저장 경로 설정
faiss_index_path = "./faiss_index.bin"
metadata_path = "./documents.pkl"

# 5️⃣ PDF 문서 리스트
# file_list = ["document1.pdf", "document2.pdf"]  # 실제 파일 경로로 변경
# path = "/your/pdf/path"  # 실제 경로로 변경

# 6️⃣ FAISS GPU 인덱스 생성
res = faiss.StandardGpuResources()  # GPU 리소스 할당
index = None

# 7️⃣ 문서 처리
all_documents = []
for file in file_list:
    file_path = os.path.join(path, file)
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
    except:
        documents = []
        with fitz.open(file_path) as pdf:
            metadata = pdf.metadata # PDF의 메타데이터 추출
            
            for page_num, page in enumerate(pdf):
                page_text = page.get_text()
                if "MuPDF error:" not in page_text:
                    documents.append(Document(
                        page_content=page_text,
                        metadata={
                            'producer': metadata.get('producer', ''),
                            'creator': metadata.get('creator', ''),
                            'creationdate': metadata.get('creationDate', ''),
                            'title': metadata.get('title', ''),
                            'author': metadata.get('author', ''),
                            'moddate': metadata.get('modDate', ''),
                            'pdfversion': metadata.get('pdfVersion', ''),
                            'source': file_path,
                            'total_pages': pdf.page_count,
                            'page': page_num,
                            'page_label': str(page_num + 1)
                        }
                    ))

    # 8️⃣ 텍스트 정제
    for doc in documents:
        doc.page_content = doc.page_content.replace("\n", " ").replace("  ", " ")
        doc.metadata["source"] = file

    # 9️⃣ 문서 스플리팅
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    split_documents = splitter.split_documents(documents)

    # 1️⃣0️⃣ 텍스트 추출 및 임베딩 생성
    texts = [doc.page_content for doc in split_documents]
    embeddings = []

    batch_size = 16
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        batch_embeddings = get_embeddings(batch)
        embeddings.extend(batch_embeddings)

    embeddings = np.array(embeddings, dtype=np.float32)
    
    # 1️⃣1️⃣ FAISS GPU 인덱스 생성 (처음이면 초기화)
    if index is None:
        embedding_dim = embeddings.shape[1]
        cpu_index = faiss.IndexFlatL2(embedding_dim)  # CPU 인덱스
        index = faiss.index_cpu_to_gpu(res, 0, cpu_index)  # GPU로 변환
    
    # 1️⃣2️⃣ FAISS 인덱스에 데이터 추가
    index.add(embeddings)
    all_documents.extend(split_documents)

    print(f"✅ {file} 처리 완료")

# 1️⃣3️⃣ FAISS 인덱스 저장
faiss.write_index(faiss.index_gpu_to_cpu(index), faiss_index_path)

# 1️⃣4️⃣ 문서 저장
with open(metadata_path, "wb") as f:
    pickle.dump(all_documents, f)

print("🎉 모든 문서 처리 및 저장 완료!")


✅ 일반_let safe 팬덤안심 상해보험.pdf 처리 완료
✅ 일반_let safe 골프보험.pdf 처리 완료
✅ 일반_let safe 주택화재보험Ⅱ.pdf 처리 완료
✅ 공동인수_영업용 자동차보험(공동).pdf 처리 완료


Ignoring wrong pointing object 147 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)


✅ 상해,질병_(무) let care 실손의료보험Ⅳ(개인 단체연계용)(2501).pdf 처리 완료
✅ 상해,질병_(무) 암보험(2502).pdf 처리 완료
✅ 이륜차_let way 이륜자동차보험.pdf 처리 완료


Ignoring wrong pointing object 237 0 (offset 0)
Ignoring wrong pointing object 239 0 (offset 0)


✅ 일반_let safe 레저상해보험.pdf 처리 완료
✅ 상해,질병_(무) let smile 종합건강보험(더끌림 포우먼)(2501) 1~2종.pdf 처리 완료
✅ 일반_let safe 시민안전보험.pdf 처리 완료
✅ 개인용_let click 개인용자동차보험(인터넷).pdf 처리 완료


...

✅ 일반_let celebrate 홀인원보험.pdf 처리 완료
✅ 상해,질병_(무) let simple 간편 암보험(2404).pdf 처리 완료
✅ 일반_let safe 공무원상해보험.pdf 처리 완료
✅ 기타_let click 배달플랫폼자동차보험(인터넷).pdf 처리 완료


Ignoring wrong pointing object 163 0 (offset 0)


✅ 제도성특약_실손의료비 보장 무사고할인 특별약관 [2011.4.1 이후 갱신도래 계약 적용].pdf 처리 완료


Ignoring wrong pointing object 155 0 (offset 0)


✅ 상해,질병_(무) let care 치아보험(2404).pdf 처리 완료
✅ 상해,질병_(무) let care 상해보험(2404) 1종.pdf 처리 완료
🎉 모든 문서 처리 및 저장 완료!
